## 計算式の木

In [39]:
import re
 
class Node:
    def __init__(self, value):
        self.value = value
        self.left = None
        self.right = None
 
 
# 演算子の優先順位を定義
def precedence(op):
    if op in ('+', '-'):
        return 1
    if op in ('*', '/'):
        return 2
    return 0
 
 
# 式をトークン化（カッコや数値、演算子を分離）
def tokenize(expression):
    return re.findall(r'\d+|\+|\-|\*|\/|\(|\)', expression)
 
 
# 中置記法の式を構文木に変換
def build_tree_from_infix(expression):
    tokens = tokenize(expression)
    values = []  # 数値や部分木を格納するスタック
    operators = []  # 演算子を格納するスタック
 
    def apply_operator():
        operator = operators.pop()
        right = values.pop()
        left = values.pop()
        node = Node(operator)
        node.left = left
        node.right = right
        values.append(node)
 
    for token in tokens:
        if token.isdigit():  # 数値
            values.append(Node(token))
        elif token in ('+', '-', '*', '/'):  # 演算子
            while operators and operators[-1] != '(' and precedence(operators[-1]) >= precedence(token):
                apply_operator()
            operators.append(token)
        elif token == '(':  # 開き括弧
            operators.append(token)
        elif token == ')':  # 閉じ括弧
            while operators and operators[-1] != '(':
                apply_operator()
            operators.pop()  # '(' を除去
 
    while operators:  # 残りの演算子を適用
        apply_operator()
 
    return values[0]
 
# 前置記法から構文木を構築
def build_tree_from_prefix(expression):
    tokens = expression.split()
    stack = []
 
    # トークンを逆順に処理
    for token in reversed(tokens):
        if token.isdigit():  # 数値はそのままスタックに追加
            stack.append(Node(token))
        else:  # 演算子の場合、右と左をポップしてノードを構築
            left = stack.pop()
            right = stack.pop()
            node = Node(token)
            node.left = left
            node.right = right
            stack.append(node)
 
    return stack.pop()  # 最後にスタックに残ったノードがルート
 
# 後置記法から構文木を構築
def build_tree_from_postfix(expression):
    tokens = expression.split()
    stack = []
 
    for token in tokens:
        if token.isdigit():  # 数値はそのままスタックに追加
            stack.append(Node(token))
        else:  # 演算子の場合、スタックから右と左をポップしてノードを構築
            right = stack.pop()
            left = stack.pop()
            node = Node(token)
            node.left = left
            node.right = right
            stack.append(node)
 
    return stack.pop()  # 最後にスタックに残ったノードがルート
 
# 構文木を中置記法に変換
def to_infix(node):
    if not node:
        return ""
    if not node.left and not node.right:  # 葉ノード
        return node.value
    left = to_infix(node.left)
    right = to_infix(node.right)
    return f"({left} {node.value} {right})"
 
 
# 構文木を前置記法に変換
def to_prefix(node):
    if not node:
        return ""
    return f"{node.value} {to_prefix(node.left)} {to_prefix(node.right)}".strip()
 
 
# 構文木を後置記法に変換
def to_postfix(node):
    if not node:
        return ""
    return f"{to_postfix(node.left)} {to_postfix(node.right)} {node.value}".strip()
 
 
# 主関数：記法変換を行う
def convert(expression, input_type, output_type):
    # 構文木を構築
    if input_type == "infix":
        tree = build_tree_from_infix(expression)
    elif input_type == "postfix":
        tree = build_tree_from_postfix(expression)
    elif input_type == "prefix":
        tree = build_tree_from_prefix(expression)
    else:
        raise ValueError("不明な入力形式です。")
 
    # 指定された形式に変換
    if output_type == "infix":
        return to_infix(tree)
    elif output_type == "prefix":
        return to_prefix(tree)
    elif output_type == "postfix":
        return to_postfix(tree)
    else:
        raise ValueError("不明な出力形式です。")
 
 
if __name__ == "__main__":
    # 入力例
    select = 3  # 1: 中置記法から変換, 2: 前置記法から変換, 3: 後置記法から変換
    prob = "3 1 - 7 2 + * 8 2 - / "  # 問題文
 
    if select == 1:
        # 中置記法から前置記法と後置記法への変換
        print("中置記法から前置記法:", convert(prob, "infix", "prefix"))
        print("中置記法から後置記法:", convert(prob, "infix", "postfix"))
    elif select == 2:
        # 前置記法から後置記法と中置記法への変換
        print("前置記法から後置記法:", convert(prob, "prefix", "postfix"))
        print("前置記法から中置記法:", convert(prob, "prefix", "infix"))
    elif select == 3:
        # 後置記法から前置記法と中置記法への変換
        print("後置記法から前置記法:", convert(prob, "postfix", "prefix"))
        print("後置記法から中置記法:", convert(prob, "postfix", "infix"))

後置記法から前置記法: / * - 3 1 + 7 2 - 8 2
後置記法から中置記法: (((3 - 1) * (7 + 2)) / (8 - 2))


In [2]:
!pip install graphviz


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## ハフマン

In [1]:
# ハフマンコードを作成する際に用いる木を描画するシステム
# Jupyter環境での使用を推奨: (!pip install graphviz)
# 使用するためにはPCにGraphvizのインストールが必須である
# 数行下のパスは個人で指定する
# 
# 使い方:
# 下の方にある frequencies の中にそれぞれの確率を少数で書き込む
import heapq
from collections import defaultdict
from graphviz import Digraph
import os


# Graphvizのインストールディレクトリを指定
os.environ["PATH"] += os.pathsep + r'C:\Program Files\Graphviz\bin'

class Node:
    def __init__(self, char, freq):
        self.char = char
        self.freq = freq
        self.left = None
        self.right = None
    
    def __lt__(self, other):
        return self.freq < other.freq

def build_huffman_tree(frequencies):
    heap = [Node(char, freq) for char, freq in frequencies.items()]
    heapq.heapify(heap)

    while len(heap) > 1:
        left = heapq.heappop(heap)
        right = heapq.heappop(heap)
        merged = Node(None, left.freq + right.freq)
        merged.left = left
        merged.right = right
        heapq.heappush(heap, merged)

    return heap[0]

def generate_codes(node, prefix="", codebook={}):
    if node is not None:
        if node.char is not None:
            codebook[node.char] = prefix
        generate_codes(node.left, prefix + "0", codebook)
        generate_codes(node.right, prefix + "1", codebook)
    return codebook

def visualize_huffman_tree(node, graph=None):
    if graph is None:
        graph = Digraph()

    if node.char is not None:
        graph.node(str(id(node)), label=f"{node.char}:{node.freq}")
    else:
        graph.node(str(id(node)), label=str(node.freq))

    if node.left is not None:
        graph.edge(str(id(node)), str(id(node.left)), label="0")
        visualize_huffman_tree(node.left, graph)

    if node.right is not None:
        graph.edge(str(id(node)), str(id(node.right)), label="1")
        visualize_huffman_tree(node.right, graph)

    return graph

frequencies = {
    'A': 0.40,
    'D': 0.15,
    'H': 0.05,
    'N': 0.05,
    'P': 0.10,
    'R': 0.25,
}

huffman_tree = build_huffman_tree(frequencies)
codes = generate_codes(huffman_tree)

tree_graph = visualize_huffman_tree(huffman_tree)
tree_graph.render('huffman_tree', view=True, format='png')

ModuleNotFoundError: No module named 'graphviz'

## 計算時間

In [37]:
# 計算時間見積もり
# 使い方:
# ℓ27 に使用する計算量をオーダー表記で記入
# ℓ28 に提示されているリストの長さとかかる時間(ms)を入力
# ℓ29 に求める時間に対するリストの長さを入力
import math

def estimate_calc_time(complexity, data, input_size):
    data_size, data_time = data
    if complexity == "O(1)":
        ratio = 1
    elif complexity == "O(n)":
        ratio = input_size / data_size
    elif complexity == "O(n^2)":
        ratio = (input_size / data_size) ** 2
    elif complexity == "O(logn)":
        ratio = math.log(input_size) / math.log(data_size)
    elif complexity == "O(nlogn)":
        ratio = (input_size * math.log(input_size)) / (data_size * math.log(data_size))
    elif complexity == "O(2^n)":
        ratio = (2 ** input_size) / (2 ** data_size)
    else:
        raise ValueError("この関数ではその計算量は想定されていません")
    
    return data_time * ratio

complexity = "O(nlogn)"
data = [100000, 50]  #[長さ, ms]
input_size = 10000 #長さ

estimated_time = estimate_calc_time(complexity, data, input_size)
print(f"長さ {input_size} のリストをソートするのにかかる時間は {estimated_time} ms です")

長さ 10000 のリストをソートするのにかかる時間は 3.9999999999999996 ms です


## マージソート

In [18]:
def merge_sort(arr):
    if len(arr) <= 1:
        return arr
    
    mid = len(arr) // 2
    left_half = merge_sort(arr[:mid])
    right_half = merge_sort(arr[mid:])
    
    return merge(left_half, right_half)

def merge(left, right):
    sorted_list = []
    while left and right:
        if left[0] <= right[0]:
            sorted_list.append(left.pop(0))
        else:
            sorted_list.append(right.pop(0))
    sorted_list.extend(left or right)
    print(sorted_list)
    return sorted_list

In [19]:
merge_sort([9, 4, 7, 3, 5, 2, 8, 1])

[4, 9]
[3, 7]
[3, 4, 7, 9]
[2, 5]
[1, 8]
[1, 2, 5, 8]
[1, 2, 3, 4, 5, 7, 8, 9]


[1, 2, 3, 4, 5, 7, 8, 9]

## クイックソート

In [22]:
def simple_qsort(lst):
      if len(lst) <= 1:
        return lst
      pivot = lst[0]
      print(pivot)

      lst1 = [x for x in lst if x < pivot]
      lst2 = [x for x in lst if x == pivot]
      lst3 = [x for x in lst if x > pivot]

      return simple_qsort(lst1) + lst2 + simple_qsort(lst3)

In [23]:
simple_qsort([9, 4, 7, 3, 5, 2, 8, 1 ])

9
4
3
2
7


[1, 2, 3, 4, 5, 7, 8, 9]

## hukasa

In [42]:
def dfs(graph, node, visited):
    if node not in visited:
        print(node)
        visited.add(node)
        for neighbor in graph[node]:
            dfs(graph, neighbor, visited)

graph = {
    'A': ['B', 'D'],
    'B': [],
    'C': ['A', 'B', 'D', 'E'],
    'D': ['G'],
    'E': ['B',],
    'F': ['C', 'E'],
    'G': ['C', 'F']
}

visited = set()
dfs(graph, 'A', visited)

A
B
D
G
C
E
F


## haba

In [11]:
from collections import deque

def bfs(graph, start):
    visited = set()
    queue = deque([start])
    visited.add(start)

    while queue:
        node = queue.popleft()
        print(node)

        for neighbor in graph[node]:
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append(neighbor)

graph = {
    'A': ['B', 'C'],
    'B': ['A', 'D', 'E'],
    'C': ['A', 'F'],
    'D': ['B'],
    'E': ['B', 'F'],
    'F': ['C', 'E']
}

bfs(graph, 'A')

A
B
C
D
E
F


## エントロピー

In [34]:
import math
def entropy(ps):
    result = 0
    for p in ps:
        result -= p * math.log2(p)
        
    return result

ps_kakuritu = [1/8, 1/8, 1/8, 1/8, 1/8, 1/8, 1/8, 1/8]

print(entropy(ps_kakuritu))

3.0


In [13]:
from collections import Counter


def MIN_PRIORITY_QUEUE(S):
    return sorted(S.items(), key=lambda x: x[1], reverse=True)


def EXTRACT_MIN(Q):
    return Q.pop()


def INSERT(Q, z):
    return Q.append(z)


def HUFFMAN(S, a):
    n = len(S)
    Q = MIN_PRIORITY_QUEUE(S)
    for i in range(n - 1):
        left = EXTRACT_MIN(Q)
        right = EXTRACT_MIN(Q)
        freq = left[1] + right[1]
        z = (left[0] + right[0], freq)
        INSERT(Q, z)
        a.append([left, "0", left[0] + right[0]])
        a.append([right, "1", left[0] + right[0]])
        Q = dict(zip([i[0] for i in Q], [i[1] for i in Q]))
        Q = MIN_PRIORITY_QUEUE(Q)
    a.append([EXTRACT_MIN(Q), "", "top"])
    return a


def PrintReslut(b):
    for i in range(len(a)):
        now = a[i][0][0]
        num = a[i][1]
        j = 0
        while a[j][2] != 'top':
            if a[i][2] == a[j][0][0]:
                num = a[j][1] + num
                i = j
                flag = 0
                for k in range(len(a)):
                    if a[k][0][0] == 'top' or a[j][2] == a[k][0][0]:
                        flag = 1
                        break
            else:
                j += 1
        if now in s:
            b.append([now, num])
    return b


def DivideS(S):
    S = list(S)
    S = Counter(S)
    S = dict(S)
    return S

input_str = input("ここに文字列データを入力: ")
s = DivideS(input_str)
a = HUFFMAN(s, [])

result = PrintReslut([])

total_counts = sum(s.values())
average_code_length = 0

convert = {}


for b in sorted(result):
    convert[b[0]] = b[1]
    symbol, code = b[0], b[1]
    print(symbol + ": " + code)
    probability = s[symbol] / total_counts
    average_code_length += probability * len(code)

print("平均符号長:", average_code_length)

ans = ""
for i in input_str:
    ans += convert[i]

print("符号化後のビット列:", ans)

ここに文字列データを入力:  GAAGAA


A: 1
G: 0
平均符号長: 1.0
符号化後のビット列: 011011


In [33]:
node1 = Node(3)
node2 = Node(5)
node3 = Node(6)
node1.next = node2
node2.next = node3
print_list(node1)

<3, 5, 6, >


In [ ]:
線形探索→O(n)
二分探索→O(logn)
Pythonのソート→O(nlogn)
選択ソート→O(n^2)
バブルソート→O(n^2)
挿入ソート→O(n^2)
クイックソート平均→O(nlogn)
クイックソート最悪→O(n^2)
マージソート→O(nlogn)
配列を用いたリストi番目の要素の読み書き→O(1)
配列を用いたリストの末尾への要素の追加と削除→O(1)
配列を用いたリストの任意の場所への要素の追加と削除→O(n)
リンクリストのi番目の要素の読み書き→O(n)
リンクリストの任意の場所への追加末尾からの要素の削除→O(1)
↑追加や削除の対象とするノードの探索には別途O(n)かかる
スタック操作→O(1)
キュー操作→O(1)
両端キュー→O(1)
リスト→O(n)
集合→O(1)
辞書→O(1)
ハッシュテーブル→O(1)